In [32]:
from itertools import combinations, product
from statistics import mode
import numpy as np

In [33]:
def generate_basis(length):
    return [list(binary)[::-1] for binary in product([0, 1], repeat=length)]

def check_zero(indices, vector):
    return int(len(indices) == 0 or np.all(np.asarray(vector)[indices] == 0))

def vectorize_check(indices, size):
    return [check_zero(indices, vec) for vec in generate_basis(size)]


def construct_rm_matrix(order, dimension):
    index_sets = get_all_combinations(order, dimension)
    return np.asarray([vectorize_check(idx, dimension) for idx in index_sets])

def generate_H(index, dimension):
    return [vec for vec in generate_basis(dimension) if check_zero(index, vec) == 1]


def complement_indices(indices, dimension):
    return [i for i in range(dimension) if i not in indices]


def generate_index_combinations(size, dimension):
    return [list(comb) for comb in combinations(range(dimension - 1, -1, -1), size)]


def get_all_combinations(order, dimension):
    combinations_list = []
    for i in range(order + 1):
        combinations_list.extend(generate_index_combinations(i, dimension))
    return combinations_list

def check_with_t(indices, target, dimension):
    return [
        int(np.array_equal(np.asarray(vec)[indices], np.asarray(target)[indices]))
        for vec in generate_basis(dimension)
    ]

In [34]:
def majority_decode(received_word, subset_H, indices, dimension):
    complementary = complement_indices(indices, dimension)
    vote_vectors = [check_with_t(complementary, vec, dimension) for vec in subset_H]
    votes = [np.dot(np.asarray(v), np.asarray(received_word)) % 2 for v in vote_vectors]
    return mode(votes)


def decode_message(received_with_error, order, dimension, generator_matrix):
    coefficients = np.zeros(generator_matrix.shape[0], dtype=int)
    all_indices = get_all_combinations(order, dimension)

    for step in range(order, -1, -1):
        current_combinations = generate_index_combinations(step, dimension)
        decoding_results = []

        for combination in current_combinations:
            H_subset = generate_H(combination, dimension)
            decoding_results.append(majority_decode(received_with_error, H_subset, combination, dimension))

        start_position = all_indices.index(current_combinations[0])

        for i, result in enumerate(decoding_results):
            coefficients[i + start_position] = result

        if step != 0:
            received_with_error = (coefficients.T @ generator_matrix + received_with_error) % 2
        else:
            received_with_error = coefficients.T @ generator_matrix % 2

        print(f"Слово после шага {order - step}: {received_with_error}")

    return received_with_error

In [35]:
r, m = 2, 4
generator_matrix = construct_rm_matrix(r, m)
print("Порождающая матрица G:\n", generator_matrix)

Порождающая матрица G:
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]]


In [36]:
original_message = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])
print("Исходное сообщение: ", original_message)

Исходное сообщение:  [0 1 0 0 0 0 0 0 0 1 0]


In [37]:
encoded_message = original_message @ generator_matrix % 2
print("Закодированное сообщение: ", encoded_message)

Закодированное сообщение:  [0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0]


In [38]:
error_matrix = np.eye(16, dtype=int)
message_with_error = (encoded_message + error_matrix[4]) % 2
print("Сообщение с ошибкой: ", message_with_error)

Сообщение с ошибкой:  [0 1 0 1 0 1 1 1 1 0 1 0 0 0 0 0]


In [39]:
decoded_message = decode_message(message_with_error, r, m, generator_matrix)
print("Декодированное сообщение: ", decoded_message)

Слово после шага 0: [1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0]
Слово после шага 1: [1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0]
Слово после шага 2: [0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0]
Декодированное сообщение:  [0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 0]


In [40]:
if np.array_equal(encoded_message, decoded_message):
    print("Закодированное и декодированное сообщения совпадают.\n")
else:
    print("Закодированное и декодированное сообщения не совпадают.\n")

Закодированное и декодированное сообщения совпадают.

